In [14]:
import nltk
import re
import gzip
import json
import collections
import tensorflow as tf
import random
import shutil
import os
import gensim
import numpy as np
from tqdm import tqdm_notebook
from intervaltree import Interval, IntervalTree

In [2]:
with gzip.open('../data/simplewiki/simplewiki-20171103.parsed.norm.json.gz', 'rt', encoding='utf-8') as f:
    wiki = json.load(f)

In [3]:
def word_tokenize(text):
    for word in nltk.word_tokenize(text):
        if word == '``' or word == "''":
            word = '"'
        for subword in re.split('([-/])', word):
            yield subword
            
def span_tokenize(text):
    return nltk.tokenize.util.align_tokens(word_tokenize(text), text)

In [4]:
all_words = []
for _, page in tqdm_notebook(wiki.items(), leave = False):
    all_words.append(list(word_tokenize(page['text'])))

HBox(children=(IntProgress(value=0, max=125947), HTML(value='')))

In [5]:
word2vec_model = gensim.models.word2vec.Word2Vec(all_words, iter = 15, size = 128)

In [22]:
embeddings = np.concatenate([
    word2vec_model.wv.syn0,
    word2vec_model.seeded_vector('<UNK>').reshape([1, -1]),
    word2vec_model.seeded_vector('<OOB>').reshape([1, -1])])

In [27]:
id_to_word = word2vec_model.wv.index2word + ['<UNK>', '<OOB>']
word_to_id = { word: id for id, word in enumerate(id_to_word) }

In [29]:
counter = collections.Counter()
for _, page in tqdm_notebook(wiki.items(), leave=False):
    for link in page['links']:
        counter[link['target']] += 1
id_to_target_2k = list(target for target, _ in counter.most_common(2000))
target_to_id_2k = dict((target, target_id) for target_id, target in enumerate(id_to_target_2k))
top_2k_targets = set(id_to_target_2k)

HBox(children=(IntProgress(value=0, max=125947), HTML(value='')))

In [30]:
def generate_examples_with_words(page, context_width, valid_link_targets):
    page_id = page['id']
    page_links = page['links']
    page_text = page['text']
    
    link_spans = IntervalTree()
    for link in page_links:
        link_spans[link['start']:link['end']] = link['target']

    word_spans = span_tokenize(page_text)
    processed_targets = set()
    for i, word_span in enumerate(word_spans):
        for target in link_spans[word_span[0]:word_span[1]]:
            if target.data not in valid_link_targets:
                continue
            if target in processed_targets:
                continue
            processed_targets.add(target)
            context = []
            link_mask = []
            for j in range(i - context_width, i + context_width + 1):
                if j < 0 or j >= len(word_spans):
                    context.append('<OOB>')
                    link_mask.append(False)
                else:
                    s = word_spans[j]
                    w = page_text[s[0]:s[1]]
                    context.append(w)
                    link_mask.append(target.overlaps(s[0], s[1]))
            yield (page_id, target.data, context, link_mask)

In [31]:
examples = []
for _, page in tqdm_notebook(wiki.items(), leave=False):
    examples.extend(generate_examples_with_words(page, 40, top_2k_targets))
random.shuffle(examples)

HBox(children=(IntProgress(value=0, max=125947), HTML(value='')))

In [32]:
dev_set_size = 20000
test_set_size = 20000

dev_set = examples[:dev_set_size]
test_set = examples[dev_set_size:dev_set_size+test_set_size]
train_set = examples[dev_set_size+test_set_size:]

len(dev_set), len(test_set), len(train_set)

(20000, 20000, 497099)

In [43]:
def convert_to_tfrecord(example):
    page_id, target, context, link_mask = example
    
    # convert target to ID
    target_id = target_to_id_2k[target]
    
    # convert context to IDs
    context_word_ids = []
    for word in context:
        word_id = word_to_id.get(word)
        if word_id is None:
            word_id = word_to_id['<UNK>']
        context_word_ids.append(word_id)

    # convert link mask to integers
    link_mask_ints = [int(l) for l in link_mask]
    
    return tf.train.Example(features = tf.train.Features(feature = {
        'page_id': tf.train.Feature(int64_list = tf.train.Int64List(value = [page_id])),
        'target_id': tf.train.Feature(int64_list = tf.train.Int64List(value = [target_id])),
        'context_word_ids': tf.train.Feature(int64_list = tf.train.Int64List(value = context_word_ids)),
        'link_mask': tf.train.Feature(int64_list = tf.train.Int64List(value = link_mask_ints))
    }))


def write_tfrecords(examples, path, batch_size = 10000):
    # remove old directory
    shutil.rmtree(path, ignore_errors = True)
    
    # make directory
    os.makedirs(path, exist_ok = True)
    
    # write batches
    for offset in tqdm_notebook(range(0, len(examples), batch_size), leave = False):
        batch = examples[offset:offset + batch_size]
        batch_path = os.path.join(path, 'examples.%010d.tfrecords.gz' % offset)
        options = tf.python_io.TFRecordOptions(tf.python_io.TFRecordCompressionType.GZIP)
        with tf.python_io.TFRecordWriter(batch_path, options = options) as writer:
            for example in batch:
                writer.write(convert_to_tfrecord(example).SerializeToString())

In [35]:
with open('../data/simplewiki/simplewiki-20171103.el_softmax_4.vocab.txt', 'wt') as f:
    for word in id_to_word:
        print(word, file = f)

In [36]:
with open('../data/simplewiki/simplewiki-20171103.el_softmax_4.targets.txt', 'wt') as f:
    for target in id_to_target_2k:
        print(target, file = f)

In [39]:
np.save('../data/simplewiki/simplewiki-20171103.el_softmax_4.embeddings.npy', embeddings)

In [44]:
write_tfrecords(dev_set, '../data/simplewiki/simplewiki-20171103.el_softmax_4.dev')
write_tfrecords(test_set, '../data/simplewiki/simplewiki-20171103.el_softmax_4.test')
write_tfrecords(train_set, '../data/simplewiki/simplewiki-20171103.el_softmax_4.train')

HBox(children=(IntProgress(value=0, max=2), HTML(value='')))

HBox(children=(IntProgress(value=0, max=2), HTML(value='')))

HBox(children=(IntProgress(value=0, max=50), HTML(value='')))